## 팀프로젝트4

### 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine, inspect
from google.cloud import storage, bigquery


### data 파일 용량 확인

In [ ]:
# 파일 용량 확인
def get_csv_file_sizes(directory):
    file_sizes = {}
    for filename in os.listdir(directory):
        if filename.endswith('.parquet'):
            file_path = os.path.join(directory, filename)
            file_size_bytes = os.path.getsize(file_path)
            file_size_mb = file_size_bytes / (1024 * 1024)
            file_sizes[filename] = file_size_mb
    return file_sizes

In [ ]:
# directory = '데이터가 저장된 위치'
directory = './origin_data'
csv_file_sizes = get_csv_file_sizes(directory)
sorted_dict = dict(sorted(csv_file_sizes.items(), key=lambda item: item[1], reverse=True))
for filename, size in sorted_dict.items():
    print(f'{filename:<50}: {size:.2f} MB')

### GCS로 부터 parquet파일 받아오기

In [ ]:
# 환경 변수에 JSON 키 파일 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./config/codeit-project-567b5092fd38.json"

# GCS 클라이언트 및 BigQuery 클라이언트 초기화
storage_client = storage.Client()
bigquery_client = bigquery.Client()

def download_parquet_from_gcs(bucket_name, prefix):
    """
    GCS에서 Parquet 파일 다운로드 및 병합.
    :param bucket_name: GCS 버킷 이름
    :param prefix: 다운로드할 경로 (GCS 버킷 내부 폴더)
    :return: 파일별 데이터프레임 딕셔너리 (key: 파일 이름, value: 데이터프레임)
    """
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)  # 지정된 경로의 파일 검색
    dataframes = {}  # 파일별 데이터프레임 저장
    file_names = []

    for blob in blobs:
        if blob.name.endswith(".parquet"):
            file_name = blob.name.split("/")[-1].replace(".parquet", "")  # 파일 이름 추출
            print(f"Downloading: {blob.name}")
            # GCS에서 바로 메모리로 읽기
            with blob.open("rb") as file:
                df = pd.read_parquet(file, engine="pyarrow")
                dataframes[file_name] = df
                file_names.append(file_name)

    if not dataframes:
        print(f"No Parquet files found at prefix: {prefix}")
    return dataframes, file_names  # 파일 이름별 데이터프레임 딕셔너리 반환


In [ ]:
## 그냥 데이터 다운로드만 하고 확인 할 때 사용.
## 실행, prefix, dataset_name은 동일하게 들어갈 것 같습니다.
if __name__ == "__main__":
    db_name = "origin/votes"
    bucket_name = "finalproject_sprint"
    prefix = db_name  # GCS 내부의 특정 경로(버킷에서 파일이 저장된 폴더 이름.)
    dataset_name = db_name  # 데이터셋 이름 지정

    # GCS에서 Parquet 데이터 다운로드
    dataframes, file_names = download_parquet_from_gcs(bucket_name, prefix)
    print(f"DB name : {db_name}, table_name : {file_names}")

    # GCS에서 받은 parquet의 file이름으로 데이터 저장
    for file_name in file_names:
        globals()[file_name] = dataframes[f'{file_name}']
    
    del dataframes

In [ ]:
accounts_userwithdraw['year_month'] = accounts_userwithdraw['created_at'].dt.strftime('%Y-%m')

In [ ]:
con = accounts_userwithdraw[accounts_userwithdraw['year_month'] == '2023-07']
con['reason'].value_counts() / con.shape[0]

In [ ]:
accounts_user[accounts_user['firebase_uid'].isna()]

def find_null_col(df):
    null_rows = df[df.isna().any(axis=1)]
    display(null_rows)

find_null_col(accounts_user)

In [ ]:
accounts_paymenthistory['productId'].value_counts()

In [ ]:
accounts_userquestionrecord.head()

In [ ]:
accounts_userquestionrecord['status'].value_counts()

### polls_question

In [ ]:
polls_question

In [ ]:
polls_question['date'] = polls_question['created_at'].dt.strftime('%Y-%m-%d')
polls_question

In [ ]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 형태소 분석 함수
def preprocess(text):
    okt = Okt()
    return okt.nouns(text) ## morphs, nouns

# 명사 추출
polls_question['nouns'] = polls_question['question_text'].apply(preprocess)

# 모든 문서의 명사 합치고 빈도수 계산
all_nouns = polls_question['nouns'].sum()
counts = Counter(all_nouns)

# 워드 클라우드 생성
path = 'C:/Users/seoho/AppData/Local/Microsoft/Windows/Fonts/NanumSquare.ttf'
wordcloud = WordCloud(font_path='malgun', background_color='white')
wordcloud_image = wordcloud.generate_from_frequencies(counts)

# 시각화
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud_image, interpolation='bilinear')
plt.axis('off')
plt.title("polls_quesition Word Cloud")

plt.savefig('wordcloud.png', dpi=600)

plt.show()

In [ ]:
polls_question.iloc[5022]

In [ ]:
from collections import Counter

# DataFrame의 'nouns' 컬럼에서 모든 명사를 하나의 리스트로 합칩니다.
all_nouns = polls_question['nouns'].sum()

# Counter를 사용하여 각 단어의 빈도를 계산합니다.
word_counts = Counter(all_nouns)

# 결과확인
print(word_counts)

In [ ]:
polls_question[polls_question['id'] == 3115]

In [ ]:
polls_questionpiece

### polls_questionset


In [ ]:
polls_questionset['opening_time'] = np.where(polls_questionset['opening_time'] < polls_questionset['created_at'], polls_questionset['created_at'], polls_questionset['opening_time'])
polls_questionset['open_diff'] = polls_questionset['opening_time'] - polls_questionset['created_at']

In [ ]:
polls_questionset['open_diff'].max()

In [ ]:
polls_questionset['date'] = polls_questionset['opening_time'].dt.strftime('%Y-%m-%d')
polls_questionset['year_month'] = polls_questionset['opening_time'].dt.to_period("M")
polls_questionset

In [ ]:
polls_questionset['year_month'].value_counts().sort_index()

### polls_questionpiece

In [ ]:
polls_questionpiece['year_month'] = polls_questionpiece['created_at'].dt.to_period("M")
polls_questionpiece['date'] = polls_questionpiece['created_at'].dt.strftime("%Y-%m-%d")
polls_questionpiece

In [ ]:
popular_question = polls_questionpiece.groupby('year_month')['question_id'].value_counts().to_frame(name='count').reset_index(level='question_id')
popular_question
# pop_quest_df = pd.DataFrame(popular_question)
# pop_quest_df

popular_question.loc['2023-04']

In [ ]:
polls_question

In [ ]:
polls_question.rename(columns={'id' : 'question_id'}, inplace=True)
polls_question

In [ ]:
year_months = popular_question.index.get_level_values(0).unique()
best_question = []

for year_month in year_months:
    loc = popular_question.loc[f'{year_month}'][:20]
    merged_df = pd.merge(loc, polls_question, on='question_id', how='left')
    merged_df['year_month'] = year_month
    filtered_data = merged_df.query(f"question_text != 'vote'").head(10)
    best_question.append(filtered_data)
    #print(question_texts)


# best_question[12]

result_df = pd.concat(best_question, ignore_index=True)
result_df.to_csv('question.csv')
result_df

In [ ]:
question_table = polls_questionpiece.groupby('question_id')['question_id'].value_counts()
top_question = question_table.sort_values(ascending=False).to_frame(name='count').reset_index(level='question_id')
top_question_df = pd.merge(top_question, polls_question, on='question_id', how='left')
top_question_df


In [ ]:
polls_question[polls_question['question_text'] == 'vote']

In [ ]:
merged_df = pd.merge(popular_question, polls_question, on='question_id', how='left')


# 병합된 데이터프레임에서 'question_text' 열 추출
question_texts = merged_df['question_text']

print(question_texts)


### events

In [ ]:
events

In [ ]:
event_receipts['event_id'].value_counts()

In [ ]:
polls_question

In [ ]:
accounts_userquestionrecord

In [ ]:
polls_usercandidate

In [ ]:
polls_questionset

In [ ]:
polls_questionpiece

In [ ]:
accounts_pointhistory[accounts_pointhistory['delta_point'] < 0 ]

In [ ]:
accounts_paymenthistory

In [ ]:
payment_df = accounts_paymenthistory.copy()

In [ ]:
payment_df['point'] = payment_df['productId'].str.split('.').str[1].astype(int)

In [ ]:
def get_first_last_datetime(series):
    return series.min(), series.max()

user_payment_df  = payment_df.groupby('user_id').agg({
    'productId' : list, 
    'created_at' : list, 
    'point' : 'sum'})

user_payment_df .columns = ['products', 'purchase_dates', 'total_purchases']


user_payment_df .reset_index(level=0, inplace=True)
user_payment_df 

In [ ]:
monthly_purchase = accounts_paymenthistory.groupby(accounts_paymenthistory['created_at'].dt.to_period("M"))['productId'].count()

# 시각화
monthly_purchase.plot(kind='bar')
plt.xlabel('Year-Month')
plt.ylabel('Purchase Count')
plt.title('Monthly Purchase Count')
plt.xticks(rotation=45)  # x축 라벨 회전 (필요에 따라 조절)
plt.show()      

### votes_accounts_user

In [ ]:
accounts_user['gender'].value_counts()

In [ ]:
accounts_user

In [ ]:
polls_questionreport

In [ ]:
polls_questionpiece

## 선기님 data 추출



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pointhistory_df = accounts_pointhistory
userquestionrecord_df = accounts_userquestionrecord
questionpiece_df = polls_questionpiece
event_receipts_df = event_receipts
question_df = polls_question

# 데이터 결합: 단계별 점검
# Step 1: accounts_pointhistory -> accounts_userquestionrecord 결합
merged_data = pointhistory_df.merge(userquestionrecord_df, left_on='user_question_record_id', right_on='id', how='left')
print(merged_data.columns)

# Step 2: accounts_userquestionrecord -> polls_questionpiece 결합
if 'question_piece_id' in merged_data.columns:  # 컬럼 존재 여부 확인
    merged_data = merged_data.merge(questionpiece_df, left_on='question_piece_id', right_on='id', how='left')
else:
    raise KeyError("`question_piece_id` 컬럼이 누락되었습니다.")

print(merged_data.columns)

# Step 3: polls_questionpiece -> polls_question 결합
merged_data.rename(columns={'question_id_x': 'question_id', 'created_at': 'question_created_at'}, inplace= True)
question_df.rename(columns={'id': 'question_id'}, inplace= True)

if 'question_id' in merged_data.columns:  # 컬럼 존재 여부 확인
    merged_data = merged_data.merge(question_df, left_on='question_id', right_on='question_id', how='left')
else:
    raise KeyError("`question_id` 컬럼이 누락되었습니다.")

# 최종 데이터 확인
print("최종 결합된 데이터:\n", merged_data.head())

# 필요한 컬럼만 선택
selected_columns = ['delta_point', 'question_id', 'has_read', 'answer_status', 'report_count', 'opened_times', 'is_voted', 'is_skipped', 'question_text']

# 선택한 컬럼으로 데이터 프레임을 재구성
selected_data = merged_data[selected_columns]

# 결과 출력
display(selected_data)

### grouped의 dataframe에서 OKT를 거친 결과

In [ ]:
# Step 1: NaN 제거
# NaN 또는 값이 없는 delta_point를 제거
filtered_data = selected_data.dropna(subset=['question_id', 'has_read', 'is_voted', 'is_skipped'])

# Step 2: 필요 없는 delta_point 값 필터링 (0인 질문 제거)
filtered_data = filtered_data[filtered_data['question_id'] != 0]

# Step 3: delta_point 값별로 주요 특징 그룹화
grouped = filtered_data.groupby('delta_point').agg({
    'question_id': 'nunique',  # 고유 질문 수
    'has_read': 'mean',  # 질문 읽기 비율
    'is_voted': 'mean',  # 질문 투표 비율
    'is_skipped': 'mean',  # 질문 스킵 비율
    'report_count': 'sum',  # 신고 횟수 합계
    'opened_times': 'sum',  # 열린 횟수 합계
    'question_text': lambda x: list(x.unique())  # 고유 질문 텍스트 리스트
}).reset_index()

# Step 4: 컬럼 이름 변경
grouped.rename(columns={
    'question_id': 'unique_questions',
    'has_read': 'avg_has_read',
    'is_voted': 'avg_is_voted',
    'is_skipped': 'avg_is_skipped',
    'report_count': 'total_reports',
    'opened_times': 'total_opened',
    'question_text': 'related_questions'
}, inplace=True)

# Step 5: 결과 출력
print("Delta Point 기준 엔터티:\n")
display(grouped)

In [ ]:
def okt_nouns(text):
    okt = Okt()
    return okt.nouns(text) ## nouns

def okt_morphs(text):
    okt = Okt()
    return okt.morphs(text) ## morphs


# 명사 추출
grouped['nouns'] = grouped['related_questions'].apply(lambda x: okt_nouns(' '.join(x)))


grouped['morphs'] = grouped['related_questions'].apply(lambda x: okt_morphs(' '.join(x)))

In [ ]:
grouped.to_csv("grouped.csv")

In [ ]:
# Step 1: 리스트를 개별 항목으로 분리
exploded_data = grouped.explode('related_questions')

# Step 2: 각 question_text가 연결된 delta_point 목록 만들기
question_delta_mapping = exploded_data.groupby('related_questions')['delta_point'].unique().reset_index()

# Step 3: delta_point 목록 길이 계산
question_delta_mapping['num_delta_points'] = question_delta_mapping['delta_point'].apply(len)

# Step 4: 같은 질문이 여러 delta_point 값에서 나타나는지 확인
repeated_questions = question_delta_mapping[question_delta_mapping['num_delta_points'] > 1]

# 결과 출력
print("같은 질문이 여러 delta_point 값에서 나타나는 경우:")
display(repeated_questions)

### polls_question Category
- 투표 관련하여 시제 및 문장 상태에 대해서 카테고리 분류
  

In [ ]:
# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

question_df = polls_question.copy()
question_df.head()

import pandas as pd
import re

# 불필요한 문자 제거 함수
def clean_text(text):
    # 따옴표 및 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = text.strip()  # 앞뒤 공백 제거
    return text

# question_text 컬럼에 적용
question_df['cleaned_question_text'] = question_df['question_text'].apply(clean_text)

# 결과 출력
print(question_df[['question_text', 'cleaned_question_text']])

# 'cleaned_question_text' 값이 'vote'인 행 삭제
question_df = question_df[question_df['cleaned_question_text'] != 'vote']

# 데이터프레임의 크기 확인
print(f"Updated dataframe shape: {question_df.shape}")

# 삭제 결과 확인
display(question_df.head())

In [ ]:
question_df['nouns'].value_counts()

In [ ]:
import re

# 의도 분류 함수
def classify_intent(text):
    if any(word in text for word in ['어떻게', '방법', '해야', '갈래', '가자', '마실']):
        return '행동 요청'
    elif any(word in text for word in ['슬프다', '행복', '기쁨', '감정', '감성', '고마운',
                                       '시원', '미워', '신경', '이성', '호감', '이쁘다', '이뻐',
                                       '좋아', '귀여워', '너', '부르면', '널', '연락', '생각']):
        return '감정 표현'
    elif any(word in text for word in ['만약', '면', '상상', '같은', '같았던', '줄 수', '미래',
                                       '싶다', '싶어', '할']):
        return '상상/가정'
    elif any(word in text for word in ['누가', '더 잘', '비교', '중요', '더 나은', '가까운',
                                       '많은', '어울리는', '가장', '제일', '왕', '엄친',
                                       '우리학교', '모태', '인간', '고인물', 'ACE', 'GOAT',
                                       '최고', '만렙']):
        return '판단/비교 요청'
    elif any(word in text for word in ['왜', '이유', '과정', '매력', '좋은', '잘하는', 'mbti',
                                       '성애자', '상', '싶은', '는', '사람', '친구', '덕후', 
                                       '트렌드', '알고보니', '파', '가까울']):
        return '설명 요청'
    else:
        return '기타'
        
def classify_tone(text):
    if any(word in text for word in ['잘', '제일', '아름다운', '멋진', '좋은', '웃', '성실', '어울리는', 
                                     '센스', '고마운', '스며드는', '풍부한', '미워할 수 없는', '예의바른',
                                     '시원', '많', '매력', '어른', '넘사벽', '듬직', '귀여운', '꼼꼼한',
                                     '이쁜', '꿇리지 않는', '나은', '대박', '잘생긴', '재밌는', '예쁜',
                                     '창의적인', '옷', '유행', '스타일', 'ACE', '트렌드', '최고', '실물파',
                                     '인간샤넬', '원탑', '만능', '성공', '완벽', '열심', '만능', '비주얼','성숙',
                                     '호감', '좋아할', '볼매', '필요', '소중', '굉장하다', '미쳤다', '엄친', '프리패스',
                                     '이쁘다']):
        return '칭찬'
    else:
        return '호기심'


#시간 분류 함수
def classify_time(text):
    if any(word in text for word in ['했던', '년 전', '돌아간다면', '것이다', '였던', '인 적', '었을', '봤',
                                     '했']):
        return '과거'
    elif any(word in text for word in ['후', '미래', '년 뒤', '나중에', '선생님이 될', '다면', '되면', '꾸릴',
                                       '수능', '2세', '년이 지나', '갈거 같은', '합격할 것', '년 안에',
                                       '뒤에', '결혼', '내년', '살 것', '20살', '성인', '대학교', '성공할 것',
                                       '100만구독자', '졸업 후', '어른이 되면', '대학가서']):
        return '미래'
    else:
        return '현재'
    
# 데이터프레임에 의도, 톤, 시간 추가
question_df['intent'] = question_df['cleaned_question_text'].apply(classify_intent)
question_df['tone'] = question_df['cleaned_question_text'].apply(classify_tone)
question_df['time_frame'] = question_df['cleaned_question_text'].apply(classify_time)

# 레이블 매핑
intent_mapping = {
    '감정 표현': 0,
    '상상/가정': 1,
    '판단/비교 요청': 2,
    '설명 요청': 3,
    '행동 요청': 4,
    '기타': 5
}

tone_mapping = {
    '칭찬': 0,
    '호기심': 1
}

time_mapping = {
    '과거': 0,
    '현재': 1,
    '미래': 2,
    '알 수 없음': 3
}

# 레이블 추가
question_df['intent_label'] = question_df['intent'].map(intent_mapping)
question_df['tone_label'] = question_df['tone'].map(tone_mapping)
question_df['time_label'] = question_df['time_frame'].map(time_mapping)

question_df = question_df[~question_df['intent_label'].isin([4, 5])]



# 결과 확인
display(question_df[['cleaned_question_text',  'tone', 'tone_label', 'intent', 'intent_label', 'time_frame', 'time_label']].head())

question_df = question_df[['question_id', 'created_at', 'cleaned_question_text','tone', 'tone_label', 'intent', 'intent_label', 'time_frame', 'time_label']] 

question_df.to_csv('./data/question_categorize.csv', encoding= 'utf-8-sig')